In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import pandas as pd
import os

In [ ]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

In [2]:
def init_download(driver, icao_code: str):
    driver.get("https://www.degreedays.net/")
    input_elem = driver.find_element(By.CSS_SELECTOR, '#formSlot > div > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(1) > input')
    input_elem.send_keys(icao_code)
    sleep(2)
    # options
    select_element = Select(driver.find_element(By.CSS_SELECTOR, '#formSlot > div > table:nth-child(5) > tbody > tr:nth-child(2) > td > div > div:nth-child(1) > table > tbody > tr:nth-child(2) > td > div > div > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(3) > td:nth-child(2) > select'))
    select_element.select_by_visible_text("Last 36 months")
    sleep(2)
    button_elem = driver.find_element(By.CSS_SELECTOR, '#formSlot > div > table:nth-child(5) > tbody > tr:nth-child(3) > td:nth-child(2) > button')
    button_elem.click()
    sleep(2)
    error_elem = driver.find_elements(By.CSS_SELECTOR, '#dataStatusWrapper > div > table > tbody > tr > td > table > tbody > tr > td:nth-child(2) > div')
    if error_elem:
        print(f'{icao_code} not found')
        driver.quit()
        return
    sleep(5)
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#dataStatusWrapper > div > table > tbody > tr > td > table > tbody > tr > td:nth-child(3) > div > button'))).click()
    sleep(5)
    print(f'{icao_code} downloaded')
    driver.quit()

In [3]:
def scrape(icao_code):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath('./output')
    })
    options.add_argument("--headless")

    driver = webdriver.Chrome(executable_path='chromedriver',
                                options=options) 

    init_download(driver, icao_code)

def set_up_threads(icao_codes, cores):
    with ThreadPoolExecutor(max_workers=cores) as executor:
        return executor.map(scrape,    
                            icao_codes,
                            timeout = 60)

In [ ]:
# run this cell to start scraping
df = pd.read_csv('icao-with-country-names.csv')
ICAO_CODES = list(df['icao'][df['icao'].notna()])
set_up_threads(ICAO_CODES, 4)